In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
data = pd.read_csv('/content/drive/My Drive/TRES.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.drop(columns='Unnamed: 9', axis=1, inplace=True)
rows_to_drop = [16, 17, 18]
data.drop(index=rows_to_drop, inplace=True)

In [ ]:
data.head()

,Wavelength,t1,t2,t3,a1,a2,a3,Emission Intensity,Normalized Emission Intensity
0,450.0,NaN,7.268,649.0,NaN,0.647,0.215,93837.27202,0.16264
1,460.0,NaN,10.035,813.0,NaN,0.531,0.296,195593.02270,0.34176
2,470.0,5.260,24.500,1035.0,0.370,0.170,0.392,328154.75350,0.57511
3,480.0,10.674,29.288,1011.7,0.275,0.155,0.475,448777.80880,0.78744
4,490.0,NaN,13.075,1000.0,NaN,0.420,0.522,527839.55200,0.92662


In [ ]:
data

,Wavelength,t1,t2,t3,a1,a2,a3,Emission Intensity,Normalized Emission Intensity
0,450.0,NaN,7.268,649.00,NaN,0.6470,0.215,93837.27202,0.16264
1,460.0,NaN,10.035,813.00,NaN,0.5310,0.296,195593.02270,0.34176
2,470.0,5.260,24.500,1035.00,0.370,0.1700,0.392,328154.75350,0.57511
3,480.0,10.674,29.288,1011.70,0.275,0.1550,0.475,448777.80880,0.78744
4,490.0,NaN,13.075,1000.00,NaN,0.4200,0.522,527839.55200,0.92662
5,500.0,NaN,31.618,1041.00,NaN,0.1730,0.690,558639.60680,0.98083
6,510.0,0.790,53.859,1200.50,-0.230,0.1223,0.740,567372.06410,0.99620
7,520.0,1.500,NaN,764.50,-0.170,NaN,0.863,545578.65870,0.95784
8,530.0,3.280,NaN,776.24,-0.236,NaN,0.891,508595.51300,0.89274
9,540.0,6.000,NaN,870.33,-0.150,NaN,0.826,449767.99280,0.78919


In [ ]:
data.fillna(0, inplace=True)

# Display the DataFrame after replacing NaN values with 0
print(data)

    Wavelength      t1      t2       t3     a1      a2     a3  \
0        450.0   0.000   7.268   649.00  0.000  0.6470  0.215   
1        460.0   0.000  10.035   813.00  0.000  0.5310  0.296   
2        470.0   5.260  24.500  1035.00  0.370  0.1700  0.392   
3        480.0  10.674  29.288  1011.70  0.275  0.1550  0.475   
4        490.0   0.000  13.075  1000.00  0.000  0.4200  0.522   
5        500.0   0.000  31.618  1041.00  0.000  0.1730  0.690   
6        510.0   0.790  53.859  1200.50 -0.230  0.1223  0.740   
7        520.0   1.500   0.000   764.50 -0.170  0.0000  0.863   
8        530.0   3.280   0.000   776.24 -0.236  0.0000  0.891   
9        540.0   6.000   0.000   870.33 -0.150  0.0000  0.826   
10       550.0   7.000   0.000   835.68 -0.220  0.0000  0.814   
11       560.0   9.000   0.000   998.76 -0.337  0.0000  0.882   
12       570.0   9.000   0.000   947.11 -0.374  0.0000  0.874   
13       580.0  10.500   0.000  1007.40 -0.380  0.0000  0.865   
14       590.0  11.000   

In [ ]:
# Function to calculate I(v, t) using the formula
def calculate_I_vt(t, t1, t2, t3, a1, a2, a3,Emission_Intensity):
    term1 = a1 * np.exp(-t / t1)
    term2 = a2 * np.exp(-t / t2)
    term3 = a3 * np.exp(-t / t3)
    numerator = term1 + term2 + term3
    denominator = a1 * t1 + a2 * t2 + a3 * t3
    I_vt = Emission_Intensity*(numerator / denominator)
    return I_vt

# Calculate I(v, t) for each row in the data
data['I_vt'] = data.apply(lambda row: calculate_I_vt(1.0, row['t1'], row['t2'], row['t3'], row['a1'], row['a2'], row['a3'], row['Emission Intensity']), axis=1)

# Split the data into input features (X) and target variable (y)
X = data[['Wavelength', 'I_vt']]
y = data['Normalized Emission Intensity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
inputs = keras.Input(shape=(2,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {test_loss:.4f}')

Epoch 1/100


<ipython-input-43-532e97c2b172>:3: RuntimeWarning: divide by zero encountered in scalar divide
  term1 = a1 * np.exp(-t / t1)
<ipython-input-43-532e97c2b172>:4: RuntimeWarning: divide by zero encountered in scalar divide
  term2 = a2 * np.exp(-t / t2)


1/1 [==============================] - 1s 828ms/step - loss: 533.2094 - val_loss: 211.4527
Epoch 2/100
1/1 [==============================] - 0s 49ms/step - loss: 342.2531 - val_loss: 156.0809
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 279.2852 - val_loss: 92.6406
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 175.8010 - val_loss: 37.0420
Epoch 5/100
1/1 [==============================] - 0s 38ms/step - loss: 81.1324 - val_loss: 21.1080
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 38.0181 - val_loss: 31.8712
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 33.2255 - val_loss: 31.9439
Epoch 8/100
1/1 [==============================] - 0s 56ms/step - loss: 27.3246 - val_loss: 18.7741
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 15.5831 - val_loss: 16.1703
Epoch 10/100
1/1 [==============================] - 0s 39ms/step - loss: 16.8468 - val_loss: 31.4485
Epoc

In [ ]:
def predict_intensity(wavelength, parameters=None):
    # Get sample data for the given wavelength
    sample_data = data[data['Wavelength'] == wavelength].iloc[0].copy()

    # Update sample data with user-provided parameters
    if parameters:
        sample_data.update(parameters)

    # Calculate intensity based on updated sample data
    I_vt = calculate_I_vt(1.0, sample_data.get('t1'), sample_data.get('t2'), sample_data.get('t3'),
                           sample_data.get('a1'), sample_data.get('a2'), sample_data.get('a3'),sample_data.get('Emission Intensity'))

    # Create DataFrame for prediction
    new_data = pd.DataFrame({'Wavelength': [wavelength], 'I_vt': [I_vt]})

    # Predict intensity using the model
    predicted_intensity = model.predict(new_data)

    return predicted_intensity[0][0]

# Example usage
print(f"Predicted intensity : {predict_intensity(480):.4f}")
print(f"Predicted intensity : {predict_intensity(480, parameters={'t1': 5}):.4f}")
print(f"Predicted intensity : {predict_intensity(480, parameters={'t1': 5, 'a2': 0.6}):.4f}")

1/1 [==============================] - 0s 23ms/step
Predicted intensity : 0.8328
1/1 [==============================] - 0s 21ms/step
Predicted intensity : 0.8005
1/1 [==============================] - 0s 22ms/step
Predicted intensity : -6.0693
